# Data Preprocessing

## Step 0: Leave this cell be so everyone knows it's from Kaggle.

In [ ]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Step 1: Try opening a single image

In [ ]:
!pwd

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

ROOT = '/kaggle/input/super-ai-engineer-2021-font-recognition'
img_demo = cv2.imread(ROOT+'/train/set1/1/image.png', 0)
fig, ax = plt.subplots(1, figsize=[10, 20])
fig.set_dpi(150)
ax.imshow(img_demo, cmap='gray')
plt.show()

## Step 2: Do adaptive thresholding

In [ ]:
cv2.ADAPTIVE_THRESH_GAUSSIAN_C

In [ ]:
cv2.THRESH_BINARY

In [ ]:
# ถ้าเกิด 1 คือ อะไร 2 คืออะไร
img_th = cv2.adaptiveThreshold(img_demo, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 11, 2)
fig, ax = plt.subplots(1, figsize=[10, 20])
fig.set_dpi(200)
ax.imshow(img_th, cmap='gray')
plt.show()

In [ ]:
# ถ้าเกิด 1 คือ อะไร 2 คืออะไร
img_th = cv2.adaptiveThreshold(img_demo, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
fig, ax = plt.subplots(1, figsize=[10, 20])
fig.set_dpi(200)
ax.imshow(img_th, cmap='gray')
plt.show()

In [ ]:
img_th.shape

## Step 3: Try accessing a .json file and its content.

In [ ]:
import json
from pprint import pprint


with open('/kaggle/input/super-ai-engineer-2021-font-recognition/train/set1/1/gt_text.json') as f:
    dict_demo = json.loads(f.read())
    pprint(dict_demo[0])
    print(dict_demo[0]['rect']['width'])

## Step 4: Try cropping the text with the given 'rect'.

In [ ]:
dict_demo[0]['rect'].values()

In [ ]:
x, y, w, h = [int(np.round(i)) for i in dict_demo[0]['rect'].values()]
img_demo_cropped = img_demo[y:y+h, x:x+w]
fig, ax = plt.subplots(1, figsize=[10, 20])
fig.set_dpi(200)
ax.imshow(img_demo_cropped, cmap='gray')
plt.show()

## Step 5: Loop the steps for a single image.

In [ ]:
ROOT = '/kaggle/input/super-ai-engineer-2021-font-recognition'
img = cv2.imread(ROOT+'/train/set1/1/image.png', 0)
img = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
ny, nx = img.shape
print(f'ny: {ny} | nx: {nx}')

# For adding some margins
PADDING = 3

with open(ROOT+'/train/set1/1/gt_text.json') as f:
    dct = json.loads(f.read())
    h_list = list()
    col = 6  # each line will have 6 items
    row = int(np.ceil(len(dct)/col))    # how many lines each will have
    fig, axes = plt.subplots(row, col, figsize=[20, 20])
    r = -1
    for i, box in enumerate(dct):
        c = i%col  # modulo python  such as  7%2 =  3R1    divide fully c=0  / divide not fully c=1,2,3,4,5
        if c==0: r+=1
        x, y, w, h = [int(np.round(i)) for i in box['rect'].values()]
        # Skip invalid boxes
        invalid = not(x<nx and x+w<nx and y<ny and y+h<ny)
        if invalid: continue
        print(i, x, y, w, h)
        img_cropped = img[y-PADDING:y+h+PADDING, x-PADDING:x+w+PADDING]
        axes[r][c].get_yaxis().set_visible(False)  #closing axis value
        axes[r][c].get_xaxis().set_visible(False)  #closing axis value
        axes[r][c].imshow(img_cropped, cmap='gray')
    plt.show()

In [ ]:
with open(ROOT+'/train/set1/1/gt_text.json') as f:
    dct = json.loads(f.read())
    print(dct)
    print(len(dct))
    for i, box in enumerate(dct):
        print('this is ', i, ' : and this is >', box)  ### i is the n of element and box is the dict of each block
        print('---->',box['parentId'])

## Step 6: Loop the previous step for all train images.

In [ ]:
import glob

# Try this loop first. We expect a to be 4000.
a = 0
for _ in glob.glob(ROOT+"/train/**/*.png", recursive=True):
    a += 1
print(a)

In [ ]:
imgs = glob.glob(ROOT+"/train/**/*.png", recursive=True)
dicts = glob.glob(ROOT+"/train/**/*.json", recursive=True)

In [ ]:
imgs[:10]

In [ ]:
# NET

PADDING = 3

list_image_cropped = list()
for fimg, fdict in zip(imgs, dicts):
    fimg = cv2.imread(fimg, 0)
    fimg = cv2.adaptiveThreshold(fimg, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 11, 2)
    with open(fdict) as f:
        dct = json.loads(f.read())
        for box in dct:
            x, y, w, h = [int(np.round(k)) for k in box['rect'].values()]
            # Skip invalid boxes
            invalid = not(x<nx and x+w<nx and y<ny and y+h<ny)
            if invalid: continue
            img_cropped = fimg[y-PADDING:y+h+PADDING, x-PADDING:x+w+PADDING]
            list_image_cropped.append(img_cropped)

In [ ]:
    dct = json.loads(f.read())
    print(dct)
    print(len(dct))
    for i, box in enumerate(dct):
        print('this is ', i, ' : and this is >', box)  ### i is the n of element and box is the dict of each block
        print('---->',box['parentId'])

In [ ]:
# NET add parent_id
PADDING = 3

list_image_cropped = list()
list_of_parent_ids = []
for fimg, fdict in zip(imgs, dicts):
    fimg = cv2.imread(fimg, 0)
    fimg = cv2.adaptiveThreshold(fimg, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 11, 2)
    with open(fdict) as f:
        dct = json.loads(f.read())
        for box in dct:
            x, y, w, h = [int(np.round(k)) for k in box['rect'].values()]
            # Skip invalid boxes
            invalid = not(x<nx and x+w<nx and y<ny and y+h<ny)
            if invalid: continue
            img_cropped = fimg[y-PADDING:y+h+PADDING, x-PADDING:x+w+PADDING]
            list_image_cropped.append(img_cropped)
            list_of_parent_ids.append(box['parentId'])

In [ ]:
print(len(list_image_cropped))
print(len(list_of_parent_ids))

In [ ]:
import pandas as pd
df_final = pd.DataFrame(
    {'parent_id': list_of_parent_ids,
     'image': list_image_cropped
    })

In [ ]:
list_image_cropped[:2]

In [ ]:
df_final

In [ ]:
df_final['image'][0][0]

In [ ]:
df_final['image'][0][0].imshow(img, cmap='gray')

In [ ]:
df_final.to_csv('final_prep_appman_data.csv', encoding='utf-8',index=False)

In [ ]:
print(len(list_image_cropped))

In [ ]:
START = 0
NUM = 10

fig, axes = plt.subplots(1, NUM, figsize=[20, 10])
for i, img in enumerate(list_image_cropped[START:START+NUM]):
    axes[i].imshow(img, cmap='gray')
plt.show()